# IGNNITION: Quick start tutorial

### <u>Problem</u>: Find the shortest path in graphs with a Graph Neural Network

Find more details on this quick-start tutorial at:
https://ignnition.net/doc/quick_tutorial/


---
# Prepare the environment

#### <u>**Note**</u>: Follow the instructions below to finish the installation

In [ ]:
#@title Installing libraries and load resources

#@markdown 1. **Hit "enter"** to complete the installation of libraries
#@markdown 2. It downloads three YAML files we will need after (train_options.yaml, model_description.yaml, global_variables.yaml)

!add-apt-repository ppa:deadsnakes/ppa
!apt-get update
!apt-get install python3.7
!python -m pip install --upgrade pip
!pip install jupyter-client==6.1.5
!pip install ignnition==1.2.2
!pip install ipython-autotime

# Download YAML files for this tutorial



In [ ]:
#@title Import libraries { form-width: "30%" }
import networkx as nx
import random
import json
from networkx.readwrite import json_graph
import os
import ignnition
%load_ext tensorboard
%load_ext autotime

In [ ]:
#@title Generate the datasets (training and validation)
import os

def generate_random_graph(min_nodes, max_nodes, min_edge_weight, max_edge_weight, p):
    while True:
        # Create a random Erdos Renyi graph
        G = nx.erdos_renyi_graph(random.randint(min_nodes, max_nodes), p)
        complement = list(nx.k_edge_augmentation(G, k=1, partial=True))
        G.add_edges_from(complement)
        nx.set_node_attributes(G, 0, 'src-tgt')
        nx.set_node_attributes(G, 0, 'sp')
        nx.set_node_attributes(G, 'node', 'entity')

        # Assign randomly weights to graph edges
        for (u, v, w) in G.edges(data=True):
            w['weight'] = random.randint(min_edge_weight, max_edge_weight)

        # Select a source and target nodes to compute the shortest path
        src, tgt = random.sample(list(G.nodes), 2)

        G.nodes[src]['src-tgt'] = 1
        G.nodes[tgt]['src-tgt'] = 1

        # Compute all the shortest paths between source and target nodes
        try:
            shortest_paths = list(nx.all_shortest_paths(G, source=src, target=tgt,weight='weight'))
        except:
            shortest_paths = []
        # Check if there exists only one shortest path
        if len(shortest_paths) == 1:
            for node in shortest_paths[0]:
                G.nodes[node]['sp'] = 1
            return nx.DiGraph(G)

def generate_dataset(file_name, num_samples, min_nodes=5, max_nodes=15, min_edge_weight=1, max_edge_weight=10, p=0.3):
    samples = []
    for _ in range(num_samples):
        G = generate_random_graph(min_nodes, max_nodes, min_edge_weight, max_edge_weight, p)
        G.remove_nodes_from([node for node, degree in dict(G.degree()).items() if degree == 0])
        samples.append(json_graph.node_link_data(G))

    with open(file_name, "w") as f:
        json.dump(samples, f)

root_dir="./data"
if not os.path.exists(root_dir):
    os.makedirs(root_dir)
if not os.path.exists(root_dir+"/train"):
    os.makedirs(root_dir+"/train")
if not os.path.exists(root_dir + "/validation"):
    os.makedirs(root_dir + "/validation")
generate_dataset("./data/train/data.json", 20000)
generate_dataset("./data/validation/data.json", 1000)

---
# GNN model configuration and training

In [ ]:
#@title Remove all the modesl previously trained (Checkpoints)
#@markdown (It is not needed to execute this the first time)
! rm -r ./CheckPoint
! rm -r ./computational_graphs

## 1) Configuration of the GNN model
### <u>Exercice</u>: Check and modify the following YAML files to configure your GNN model:
* /content/model_description.yaml -> GNN model description
* /content/train_options.yaml -> Configuration of training parameters

#### **Example**:
Try to define an optimizer with learning rate decay and set the number of samples and epochs adding the following lines in the train_options.yaml file:
```
optimizer:
  type: Adam
  learning_rate:  # define a schedule
    type: ExponentialDecay
    initial_learning_rate: 0.001
    decay_steps: 10000
    decay_rate: 0.5
...
batch_size: 1
epochs: 150
epoch_size: 200
```

In this example, there are a total of 30,000 training samples:

1 sample/step * 200 steps/epoch * 150 epochs = 30.000 samples

## 2) Load TensorBoard and execute the training

In [ ]:
#@title Load TensorBoard to visualize the evolution of learning metrics along training
#@markdown **IMPORTANT NOTE**: Click on "settings" in the TensorBoard GUI and check the option "Reload data" to see the evolution in real time. Note you can set the time interval (in seconds) to reload data.
from tensorboard import notebook
notebook.list() # View open TensorBoard instances

dir="./CheckPoint"
if not os.path.exists(dir):
    os.makedirs(dir)
%tensorboard --logdir $dir

# Para finalizar instancias anteriores de TensorBoard
# !kill 2953
# !ps aux

In [ ]:
#@title Running the GNN model training
import ignnition

def main():
    model = ignnition.create_model(model_dir= './')
    model.computational_graph()
    model.train_and_validate()
    #model.predict()


if __name__ == "__main__":
        main ()


---
# Make predictions
## (This can be only excuted once the training is finished or stopped)
#### Modify and **uncomment (\#)** the "load_model_path" variable in /content/train_options.yaml. You need to add the full path to the model checkpoint you want to load ('.hdf5' file)*

### **Note**: 
#### 'hdf5' chekpoints are saved by default as:

 /content/CheckPoint/\<experiment_date_time>/cpkt/weights.\<train_step>.\<loss>.hdf5

Note that <train_step> stands for the number of training nstept for a given checkpoint. In enables to identify the checkpoint you want to load.

#### * You may use the option "copy path" in the file menu of Colab to get the full path and use it to fill in the train_options.yaml file (click botón derecho sobre el archivo .hdf5)

In [ ]:
#@title Load functions to generate random graphs and print them
import os
import networkx as nx
import matplotlib.pyplot as plt
import json
from networkx.readwrite import json_graph
import ignnition
import numpy as np
import random
%load_ext autotime

def generate_random_graph(min_nodes, max_nodes, min_edge_weight, max_edge_weight, p):
    while True:
        # Create a random Erdos Renyi graph
        G = nx.erdos_renyi_graph(random.randint(min_nodes, max_nodes), p)
        complement = list(nx.k_edge_augmentation(G, k=1, partial=True))
        G.add_edges_from(complement)
        nx.set_node_attributes(G, 0, 'src-tgt')
        nx.set_node_attributes(G, 0, 'sp')
        nx.set_node_attributes(G, 'node', 'entity')

        # Assign randomly weights to graph edges
        for (u, v, w) in G.edges(data=True):
            w['weight'] = random.randint(min_edge_weight, max_edge_weight)

        # Select the source and target nodes to compute the shortest path
        src, tgt = random.sample(list(G.nodes), 2)

        G.nodes[src]['src-tgt'] = 1
        G.nodes[tgt]['src-tgt'] = 1

        # Compute all the shortest paths between source and target nodes
        try:
            shortest_paths = list(nx.all_shortest_paths(G, source=src, target=tgt,weight='weight'))
        except:
            shortest_paths = []
        # Check if there exists only one shortest path
        if len(shortest_paths) == 1:
            if len(shortest_paths[0])>=3 and len(shortest_paths[0])<=5:
                for node in shortest_paths[0]:
                    G.nodes[node]['sp'] = 1
                return shortest_paths[0], nx.DiGraph(G)

def print_graph_predictions(G, path, predictions,ax):
    predictions = np.array(predictions)
    node_border_colors = []
    links = []
    for i in range(len(path)-1):
        links.append([path[i], path[i+1]])
        links.append([path[i+1], path[i]])

    # Add colors to node borders for source and target nodes
    for node in G.nodes(data=True):
        if node[1]['src-tgt'] == 1:
            node_border_colors.append('red')
        else:
            node_border_colors.append('white')
    # Add colors for predictions [0,1]
    node_colors = predictions

    # Add colors for edges
    edge_colors = []
    for edge in G.edges(data=True):
        e=[edge[0],edge[1]]
        if e in links:
            edge_colors.append('red')
        else:
            edge_colors.append('black')
    pos= nx.shell_layout(G)
    vmin = node_colors.min()
    vmax = node_colors.max()
    vmin = 0
    vmax = 1
    cmap = plt.cm.coolwarm
    nx.draw_networkx_nodes(G, pos=pos, node_color=node_colors, cmap=cmap, vmin=vmin, vmax=vmax,
                           edgecolors=node_border_colors, linewidths=4, ax=ax)
    nx.draw_networkx_edges(G, pos=pos, edge_color=edge_colors, arrows=False, ax=ax, width=2)
    nx.draw_networkx_edge_labels(G, pos=pos, label_pos=0.5, edge_labels=nx.get_edge_attributes(G, 'weight'), ax=ax)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
    sm.set_array([])
    plt.colorbar(sm, ax=ax)

def print_graph_solution(G, path, predictions,ax, pred_th):
    predictions = np.array(predictions)
    node_colors = []
    node_border_colors = []
    links = []
    for i in range(len(path)-1):
        links.append([path[i], path[i+1]])
        links.append([path[i+1], path[i]])

    # Add colors on node borders for source and target nodes
    for node in G.nodes(data=True):
        if node[1]['src-tgt'] == 1:
            node_border_colors.append('red')
        else:
            node_border_colors.append('white')

    # Add colors for predictions Blue or Red
    cmap = plt.cm.get_cmap('coolwarm')
    dark_red = cmap(1.0)
    for p in predictions:
        if p >= pred_th:
            node_colors.append(dark_red)
        else:
            node_colors.append('blue')

    # Add colors for edges
    edge_colors = []
    for edge in G.edges(data=True):
        e=[edge[0],edge[1]]
        if e in links:
            edge_colors.append('red')
        else:
            edge_colors.append('black')
    pos= nx.shell_layout(G)
    nx.draw_networkx_nodes(G, pos=pos, node_color=node_colors, edgecolors=node_border_colors, linewidths=4, ax=ax)
    nx.draw_networkx_edges(G, pos=pos, edge_color=edge_colors, arrows=False, ax=ax, width=2)
    nx.draw_networkx_edge_labels(G, pos=pos, label_pos=0.5, edge_labels=nx.get_edge_attributes(G, 'weight'), ax=ax)

def print_input_graph(G, ax):
    node_colors = []
    node_border_colors = []

    # Add colors to node borders for source and target nodes
    for node in G.nodes(data=True):
        if node[1]['src-tgt'] == 1:
            node_border_colors.append('red')
        else:
            node_border_colors.append('white')

    pos= nx.shell_layout(G)
    nx.draw_networkx_nodes(G, pos=pos, edgecolors=node_border_colors, linewidths=4, ax=ax)
    nx.draw_networkx_edges(G, pos=pos, arrows=False, ax=ax, width=2)
    nx.draw_networkx_edge_labels(G, pos=pos, label_pos=0.5, edge_labels=nx.get_edge_attributes(G, 'weight'), ax=ax)

In [ ]:
#@title Load a trained model and make predictions on random graphs
#@markdown **NOTE**: You should check you can read the message "Restoring from \<load_model_path\>" at the beginning of the execution.

#@markdown Otherwise, there should be a failure to load the trained model. In this case, please check that the variable "load_model_path" in /content/train_options.yaml points to the correct path.

dataset_samples = []
sh_path, G = generate_random_graph(min_nodes=8, max_nodes=12, min_edge_weight=1, max_edge_weight=10, p=0.3)
graph = G.to_undirected()
dataset_samples.append(json_graph.node_link_data(G))

# write prediction dataset
root_dir="./data"
if not os.path.exists(root_dir):
    os.makedirs(root_dir)
if not os.path.exists(root_dir+"/test"):
    os.makedirs(root_dir+"/test")
with open(root_dir+"/test/data.json", "w") as f:
    json.dump(dataset_samples, f)

# Load the model and make predictions
model = ignnition.create_model(model_dir='./')
predictions = model.predict()

# Print the results
fig, axes = plt.subplots(nrows=1, ncols=3)
ax = axes.flatten()

# Print input graph
ax1 = ax[0]
ax1.set_title("Input graph")
print_input_graph(graph, ax1)

# Print graph with predictions (soft values)
ax1 = ax[1]
ax1.set_title("GNN predictions (soft values)")
print_graph_predictions(graph, sh_path, predictions[0], ax1)

# Print solution of the GNN
pred_th = 0.5
ax1 = ax[2]
ax1.set_title("GNN solution (p >= "+str(pred_th)+")")
print_graph_solution(graph, sh_path, predictions[0], ax1, pred_th)

# Show plot in full screen
plt.rcParams['figure.figsize'] = [10, 4]
plt.rcParams['figure.dpi'] = 100
plt.show()
